In [1]:
import pickle

with open('docs.pickle', 'rb') as f:
    docs = pickle.load(f)
len(docs)

461

In [2]:
from bs4 import BeautifulSoup

In [3]:
import base64

img_counter = 0

def save_image(base64_string, file_name):
    # Удаляем префикс "data:image/png;base64," если он есть
    if 'base64,' in base64_string:
        base64_string = base64_string.split('base64,')[1]
    
    # Декодируем base64 в бинарные данные
    image_data = base64.b64decode(base64_string)
    
    # Сохраняем изображение
    with open(f"./images/{file_name}", 'wb') as file:
        file.write(image_data)



def absolute_href(soup):
    global img_counter
    
    a_tags = soup.find_all('a')

    # Проверяем, найден ли тег
    for a_tag in a_tags:
        href = a_tag.get('href')
        if href.startswith('/'):
            a_tag['href'] = f"https://www.rustore.ru{href}"

    images = soup.find_all('img')

    for img in images:
        src = img.get('src')

        if src.startswith('/'):
            img['src'] = f"https://www.rustore.ru{src}"
        elif src.startswith('data:'):
            save_image(src, f"{img_counter:03}.png")
            img['src'] = f"/images/{img_counter:03}.png"

            img_counter += 1

    return soup

In [4]:
attr_exceptions = {
    'img': ['src', 'alt'],
    'a': ['href'],
    '*': ['id']
}

def remove_attrs(html_content):
    global attr_exceptions

    if isinstance(html_content, str):
        # Создаем объект BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
    else:
        soup = html_content

    
    # Получаем список атрибутов, которые нужно сохранить для всех тегов
    global_keep_attrs = attr_exceptions.get('*', [])
    
    # Находим все теги в документе
    for tag in soup.find_all():
        # Получаем имя тега
        tag_name = tag.name
        
        # Получаем список атрибутов для сохранения (если есть)
        keep_attrs = attr_exceptions.get(tag_name, []) + global_keep_attrs
        
        # Создаем список атрибутов для удаления
        attrs_to_remove = [attr for attr in tag.attrs if attr not in keep_attrs]
        
        # Удаляем атрибуты
        for attr in attrs_to_remove:
            del tag[attr]
    
    # Возвращаем обновленный HTML
    return soup

In [5]:
def convert_tabs_to_sequential(soup):
    # Создаем объект BeautifulSoup
    # soup = BeautifulSoup(html_code, 'html.parser')

    tabs_containers = soup.find_all('div', class_='tabs-container')

    for tab_container in tabs_containers:

        # Находим все вкладки
        tabs = tab_container.find_all('li', class_='tabs__item')

        # Находим все панели вкладок
        tab_panels = tab_container.find_all('div', attrs={'role': 'tabpanel'})

        # Создаем новый div для последовательного контента
        sequential_content = soup.new_tag('div', class_='sequential-content')

        # Добавляем заголовки и содержимое каждой вкладки последовательно

        max_panel_length = -1

        for tab, panel in zip(tabs, tab_panels):
            max_panel_length = max(max_panel_length, len(str(panel)))

        for tab, panel in zip(tabs, tab_panels):
            # Создаем заголовок
            header = soup.new_tag('h2' if max_panel_length > 300 else 'h3')
            header.string = tab.text.strip()
            sequential_content.append(header)
            
            # Добавляем содержимое вкладки
            sequential_content.append(panel)

        # Заменяем исходный контейнер вкладок на последовательный контент
        tab_container.replace_with(sequential_content)

        # Удаляем атрибут hidden из всех панелей
        for panel in sequential_content.find_all('div', attrs={'role': 'tabpanel'}):
            del panel['hidden']

    return soup

In [6]:
from bs4 import BeautifulSoup

def split_html(soup):
    # soup = BeautifulSoup(html_content, 'html.parser')

    # Находим контейнер
    container = soup.find()

    # Находим все теги h1 и h2
    headers = container.find_all(['h1', 'h2'])

    # Создаем список для хранения частей
    parts = []

    # Проходим по всем заголовкам
    for i, header in enumerate(headers):
        # Если это первый заголовок (h1)
        if i == 0:
            # Собираем все элементы до следующего h2
            content = [header]
            for sibling in header.next_siblings:
                if sibling.name == 'h2':
                    break
                content.append(sibling)
            parts.append(''.join(str(elem) for elem in content))
        else:
            # Для h2 заголовков
            content = [header]
            for sibling in header.next_siblings:
                if sibling.name == 'h2':
                    break
                content.append(sibling)
            parts.append(''.join(str(elem) for elem in content))

    # Выводим результат
    return parts

In [7]:
from bs4 import BeautifulSoup

def replace_tables_with_text(soup, title="Строка таблицы"):
    for table in soup.find_all('table'):
        # Получаем заголовки столбцов
        headers = [header.text.strip() for header in table.find_all('th')]
        
        # Если заголовки не найдены, используем первую строку как заголовки
        if not headers:
            headers = [cell.text.strip() for cell in table.find('tr').find_all(['td', 'th'])]
        
        # Создаем новый элемент div для замены таблицы
        table_replacement = soup.new_tag('div')
        
        # Обрабатываем каждую строку таблицы
        for row in table.find_all('tr')[1:]:  # Пропускаем первую строку, если она содержит заголовки
            cells = row.find_all(['td', 'th'])
            
            # Создаем заголовок для строки
            row_header = soup.new_tag('h2')
            row_header.string = title
            table_replacement.append(row_header)
            
            # Добавляем информацию о каждом столбце
            for header, cell in zip(headers, cells):
                p = soup.new_tag('p')
                p.string = f"{header}: {cell.text.strip()}"
                table_replacement.append(p)
        
        # Заменяем таблицу на новый div
        table.replace_with(table_replacement)
    
    return soup

In [8]:
from tqdm.auto import tqdm

html_splits = []
texts = set()

for doc in tqdm(docs):
    html = doc['html']
    html = html.replace("\x00", "")
    url = doc['url']
    section = " > ".join(doc["breadcrumbs"])

    soup = BeautifulSoup(html,  'html.parser')

    page_title = soup.find('h1').text.strip()

    # [table.extract() for table in soup.find_all('table')]
    soup = replace_tables_with_text(soup, section)
    soup = BeautifulSoup(str(soup),  'html.parser')

    soup  = convert_tabs_to_sequential(soup)

    # Находим все теги <p>, содержащие <strong>
    for p in soup.find_all('p'):
        strong = p.find('strong')
        if strong and strong.parent == p and len(p.contents) == 1:
            # Создаем новый тег <h2>
            h2 = soup.new_tag('h2')
            h2.string = strong.text
            # Заменяем <p><strong> на <h2>
            p.replace_with(h2)
    
    soup = remove_attrs(soup)

    soup = absolute_href(soup)

    parts = split_html(soup)


    for i, part in enumerate(parts):
        soup = BeautifulSoup(part,  'html.parser')

        h2 = soup.find('h2')

        if h2:
            title = h2.text.strip()
            a = h2.find('a')
            if not (a is None):
                a.decompose()
        else:
            title = page_title

        text = soup.get_text()

        if text in texts:
            continue

        texts.add(text)

        root = soup.find()
        tag_id = root.get('id')
        if tag_id:
            anchor = f"#{tag_id}"
        else:
            anchor = ""

        if (len(part) - len(page_title)) < 300:
            continue

        html_splits.append({
            "page_title": page_title,
            "title": title,
            "section": section,
            "html": str(soup),
            "text": text,
            "url": url + anchor 
        })
len(html_splits)

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 461/461 [00:18<00:00, 24.69it/s] 


2228

In [10]:
with open("html_splits.pickle", 'wb') as file:
    pickle.dump(html_splits, file)